In [3]:
import json
import pandas as pd
from evidently import Report
from evidently.presets import DataDriftPreset

# === Chemins relatifs depuis notebook/ ===
LOG_PATH = "../logs/predictions_log.jsonl"
REFERENCE_PATH = "../datasets/train_final.csv"
REPORT_PATH = "../logs/drift_report.html"

# === Chargement des données ===

# Données de référence
df_ref = pd.read_csv(REFERENCE_PATH)

# Sélection des colonnes pertinentes pour le modèle (les features que tu exposes dans l'API)
# Ex. ici on garde uniquement les colonnes communes aux deux datasets ensuite

# === Lecture des prédictions enregistrées dans les logs ===
records = []
with open(LOG_PATH, "r") as f:
    for line in f:
        log = json.loads(line)
        row = log.get("input", {})
        row["duration"] = log.get("duration", None)
        row["prediction"] = log.get("prediction", None)
        records.append(row)

# DataFrame des données en production
df_prod = pd.DataFrame(records)

# === Préparation des données ===
# Pour éviter les erreurs, on aligne les colonnes entre les deux DataFrames
# On garde uniquement les colonnes communes
common_columns = list(set(df_ref.columns) & set(df_prod.columns))
df_ref_filtered = df_ref[common_columns].copy()
df_prod_filtered = df_prod[common_columns].copy()

# === Analyse du drift ===
report = Report([DataDriftPreset(method="psi")])
my_rep = report.run(reference_data=df_ref_filtered, current_data=df_prod_filtered)
my_rep.save_html(REPORT_PATH)

print(f"✅ Rapport de dérive généré : {REPORT_PATH}")

✅ Rapport de dérive généré : ../logs/drift_report.html


Analyse du Drift des Données

Dans le cadre de la surveillance post-déploiement du modèle, une analyse de dérive des données a été réalisée à l’aide de la bibliothèque Evidently, afin d’identifier d’éventuels écarts significatifs entre les données d'entraînement (référence) et les données utilisées en production via l’API.

Le rapport généré (au format HTML) a été sauvegardé dans le dossier logs/ pour consultation visuelle simplifiée.

Résultats globaux

L’analyse a mis en évidence une dérive importante sur l’ensemble des 11 variables utilisées comme entrées dans l’API. Le dataset drift global est détecté avec un taux de colonnes dérivées de 100 % (11/11). Cela signifie que la distribution des variables observée en production diffère fortement de celle observée lors de l’entraînement.

Le seuil de détection configuré est basé sur l’indice PSI (Population Stability Index) : une valeur supérieure à 0.25 indique un drift significatif. Dans notre cas, toutes les variables dépassent largement ce seuil.


Parmi les variables les plus impactées par la dérive, on peut citer :

CODE_GENDER_F et CODE_GENDER_M (variables catégorielles liées au genre)

FLAG_DOCUMENT_3 (variable binaire indiquant la présence d’un document)

Ces variables présentent un écart considérable de distribution entre les phases d’entraînement et de production, comme l’illustrent les histogrammes présents dans le rapport.

Interprétation et causes potentielles

Cette dérive s’explique probablement par le caractère artificiel des données de production utilisées dans le cadre de ce projet. En effet, les inputs ont été générés ou saisis de manière manuelle dans l’interface Gradio, souvent dans le but de tester des cas spécifiques. Ces entrées ne reflètent donc pas nécessairement la réalité statistique des données que l’on retrouverait dans un contexte de production réel.

Il est important de noter que ce type de dérive ne remet pas en cause la performance du modèle lui-même, mais souligne l’importance de bien calibrer les jeux de données de monitoring. Dans un environnement réel, il conviendra de :

- Connecter l’API à des flux de données plus représentatifs,

- Mettre en place une alerte automatique lorsque la dérive dépasse un seuil critique,

- Réentraîner le modèle si la dérive persiste dans le temps ou affecte ses performances.